# Import dependencies

In [1]:
import os
import sys
import pandas as pd
import re
import pickle
import ast

# Obtener la ruta del directorio padre
parent_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
# Añadir la ruta del directorio padre al path de Python
sys.path.append(parent_path)
from connect import *
from dataStructure import *

# Read data

## Read source excel

In [13]:
PATH_EXCEL = os.path.join(parent_path,'rawData','TES3Z.xlsx')
df_excel = pd.read_excel(PATH_EXCEL,index_col=0)

In [15]:
data_path = os.path.join(parent_path,'OutputData')
files = os.listdir(data_path)
file = [f for f in files if 'df_raw' in f][0]
file_list = [f for f in files if 'list' in f][0]
file_dict = [f for f in files if 'dict' in f][0]
df_file = os.path.join(data_path,file)
list_path = os.path.join(data_path,file_list)
dict_path = os.path.join(data_path,file_dict)

with open(df_file, 'rb') as handle:
    df = pickle.load(handle)

with open(list_path, 'rb') as handle:
    list_data = pickle.load(handle)

with open(dict_path, 'rb') as handle:
    dict_data = pickle.load(handle)


# Preparing Data

## New document

In [16]:
new_tes = []
for n, row in df.iterrows():
    var = Tes(
        tesName = row['tesName'],
        tags = list(row['tags']),
        fabrication = Fabrication(
            wafer = Wafer(
                waferName = row['tesName'].split('_')[0],
                material = 'Si',
                orientation = '100',
                divisions = ['Q1','Q2','Q3','Q4'],
                chipsOnWafer= [f'Chip_{i}' for i in range(23,77)],
                image = os.path.join(parent_path,'Images','wafer_3Z.jpg')
                ),
            absorber = None if row['absorber'] == 'no' else [Layer(
                    material = 'Au',
                    process = 'electroplating',
                    orintation = 'amorphous',
                    thickness = PairValueUnit(quantity = 2, units = 'um'),
                    width = PairValueUnit(quantity = 240, units = 'um'),
                    length = PairValueUnit(quantity = 240, units = 'um')
                ),Layer(
                    material = 'Bi',
                    process = 'electroplating',
                    orintation = 'amorphous',
                    thickness = PairValueUnit(quantity = 3.5, units = 'um'),
                    width = PairValueUnit(quantity = 240, units = 'um'),
                    length = PairValueUnit(quantity = 240, units = 'um')
                )],
            membrane = None if row['membrane'] == 'no' else Layer(
                    material = 'Si3N4',
                    process = 'Sputtering',
                    orintation = 'amorphous',
                    thickness = PairValueUnit(quantity = 0.5, units = 'um'),
                    width = PairValueUnit(quantity = 250, units = 'um'),
                    length = PairValueUnit(quantity = 250, units = 'um')
                ),
            pads = [Pads(
                material = 'Mo',
                orientation = 'amorphous',
                process = 'sputtering-dc',
                thickness = PairValueUnit(quantity = 15, units = 'um') 
            ),Pads(
                material = 'Nb',
                orientation = 'amorphous',
                process = 'sputtering-dc',
                thickness = PairValueUnit(quantity = 0.15, units = 'um') 
            )],
            stack = [Layer(
                    material = 'Mo',
                    process = 'sputtering-dc',
                    orintation = 'amorphous',
                    thickness = PairValueUnit(quantity = row['stackMoT'], units = 'nm'),
                    width = PairValueUnit(quantity = row['stackw'], units = 'um'),
                    length = PairValueUnit(quantity = row['stackl'], units = 'um')
                ),Layer(
                    material = 'Au',
                    process = 'sputtering-dc',
                    orintation = 'amorphous',
                    thickness = PairValueUnit(quantity = row['stackAu1T'], units = 'nm'),
                    width = PairValueUnit(quantity = row['stackw'], units = 'um'),
                    length = PairValueUnit(quantity = row['stackl'], units = 'um')
                ),Layer(
                    material = 'Au',
                    process = 'e-beam_evaporation',
                    orintation = 'amorphous',
                    thickness = PairValueUnit(quantity = row['stackAu2T'], units = 'nm'),
                    width = PairValueUnit(quantity = row['stackw'], units = 'um'),
                    length = PairValueUnit(quantity = row['stackl'], units = 'um')
                )]
        )
    ).dict()
    new_tes.append(var)

In [17]:
faltandatos = []
for n, row in df.iterrows():
    for j, caracteristicas in enumerate(row['caracteristicas']):
        for loop in caracteristicas:
            try:
                if row['tags'][1] == 'Funcional':
                    a = dict_data[row['tesName']]
            except Exception as e:
                #print(f'Se produce error {e} en la fila {n}')
                if row['tesName'] not in faltandatos:
                    faltandatos.append(row['tesName'])


In [18]:
faltandatos

['3Z10_74_41']

In [6]:
faltan = []
for i in dict_data.keys():
    if len(df[df['tesName']==i]) == 0 and i not in faltan:
        faltan.append(i)
        


In [7]:
faltan

[]

In [49]:
for n, row in df['caracteristicas'].iterrrows():
    TCmeas(
        fromFile = list_data[0]['dataframe_fileName'],
        cryostat = df[df['tesName']==list_data[0]['tesName'][0]]['cryo'].values[0],
        holder = df[df['tesName']==list_data[0]['tesName'][0]]['holder'].values[0]
    )

0        3Z2_74_21
1        3Z2_53_13
2        3Z2_52_12
3        3Z4_64_32
4        3Z4_53_13
5        3Z4_73_33
6        3Z4_42_21
7        3Z6_73_33
8     3Z6_54_Mo/Au
9        3Z6_42_21
10       3Z6_54_34
11       3Z6_45_11
12       3Z8_44_14
13       3Z8_26_22
14       3Z8_53_13
15       3Z8_64_41
16       3Z8_73_12
17       3Z8_74_21
18       3Z8_55_44
19       3Z8_64_34
20       3Z9_45_21
21       3Z9_54_12
22       3Z9_73_32
23       3Z9_74_21
24       3Z9_54_24
25      3Z10_74_33
26      3Z10_64_41
27      3Z10_53_33
28      3Z10_33_34
29      3Z10_73_21
30      3Z10_74_41
31      3Z10_64_14
32      3Z10_53_13
33       3Z8_73_14
34       3Z8_64_32
35       3Z8_53_23
36       3Z2_54_31
37       3Z2_52_14
38       3Z2_53_33
39       3Z2_63_13
40       3Z2_63_33
41       3Z2_64_34
42       3Z2_54_34
43       3Z2_74_41
44       3Z2_64_32
45       3Z2_53_13
46       3Z2_54_24
47       3Z2_53_22
48       3Z2_63_22
49       3Z2_64_33
50       3Z2_44_24
Name: tesName, dtype: object

# Pushing data

In [6]:
objeto = DatabaseTes(True)
objeto.connect()
objeto.collection('tes')
db = objeto.collec

## Insert new documents

In [7]:
db.insert_many(new_tes)